In [ ]:
# src/finance_ml/entity/config_entity.py

@dataclass(frozen=True)
class ModelPredictionConfig:
    root_dir: Path
    trained_model_path: Path
    input_data_path: Path
    predictions_file_name: str

In [ ]:
#config/config.yaml

model_prediction:
  root_dir: artifacts/model_prediction
  trained_model_path: artifacts/model_training/lstm_model.keras
  input_data_path: artifacts/data_transformation/raw_data.csv # Path to the input data for prediction
  predictions_file_name: predictions.csv

In [ ]:
# src/finance_ml/config/configuration.py

from finance_ml.constants import *
from finance_ml.utils.common import read_yaml, create_directories
from finance_ml.entity.config_entity import DataIngestionConfig, DataTransformationConfig, ModelTrainingConfig, ModelTrainingParams, ModelEvaluationConfig, ModelPredictionConfig # Import ModelPredictionConfig
import os

class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_prediction_config(self) -> ModelPredictionConfig: # Add the new method
        config = self.config.model_prediction

        create_directories([config.root_dir])

        model_prediction_config = ModelPredictionConfig(
            root_dir=config.root_dir,
            trained_model_path=Path(config.trained_model_path), # Ensure Path type
            input_data_path=Path(config.input_data_path), # Ensure Path type
            predictions_file_name=config.predictions_file_name
        )

        return model_prediction_config

In [ ]:
# src/finance_ml/component/model_prediction.py

import os
import numpy as np
import pandas as pd
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler # Assuming you used MinMaxScaler in data transformation
from pathlib import Path

# Assuming these imports are available from previous steps
from finance_ml.entity.config_entity import ModelPredictionConfig
# Assuming a save_csv utility function exists
from finance_ml.utils.common import save_csv # Assuming a save_csv utility exists

class ModelPrediction:
    def __init__(self, config: ModelPredictionConfig):
        self.config = config

    def predict(self):
        # Load the trained model
        model = keras.models.load_model(self.config.trained_model_path)

        # Load the input data for prediction
        # Assuming the input data is a CSV with a 'close' column, similar to training data
        data = pd.read_csv(self.config.input_data_path)

        # --- Data Preprocessing for Prediction ---
        # This should mirror the preprocessing steps used for the training data
        # Assuming the prediction is based on the 'close' price and a sliding window approach (like in rorWPcAaocTq)

        # Select the 'close' column
        stock_close = data.filter(["close"])
        dataset = stock_close.values # Convert to numpy array

        # Apply the same scaler used for training data
        # IMPORTANT: You need to save and load the *fitted* scaler from the data transformation stage.
        # For this example, I'll create a new scaler, but in a real project, load the saved scaler.
        scaler = MinMaxScaler(feature_range=(0, 1)) # Use MinMaxScaler if that was used in transformation
        scaled_data = scaler.fit_transform(dataset) # Fit and transform - in a real project, just transform using loaded scaler

        # Create the input sequence for prediction (sliding window)
        # Assuming the prediction is for the next step based on the last 'time_steps' data points
        # Need to determine the 'time_steps' used during training (e.g., 60)
        time_steps = 60 # Assuming 60 based on the training example

        # Use the last 'time_steps' data points from the scaled data
        X_predict = scaled_data[-time_steps:].reshape(1, time_steps, 1) # Reshape for LSTM: (samples, time_steps, features)


        # Make the prediction
        predicted_price_scaled = model.predict(X_predict)

        # Inverse transform the prediction to get the actual price
        predicted_price = scaler.inverse_transform(predicted_price_scaled)

        # --- Save the Prediction ---
        # You might want to save the prediction along with the corresponding date or index
        # For simplicity, let's just save the predicted price
        predictions_df = pd.DataFrame(predicted_price, columns=['predicted_close_price'])

        predictions_file_path = Path(self.config.root_dir) / self.config.predictions_file_name
        save_csv(predictions_file_path, predictions_df) # Assuming a save_csv utility function exists

        print(f"Predicted next closing price: {predicted_price[0][0]}")
        print(f"Predictions saved to: {predictions_file_path}")

In [ ]:
# src/finance_ml/pipeline/stage_05_model_prediction.py

# Assuming ConfigurationManager and ModelPrediction are importable from src.finance_ml
from src.finance_ml.config.configuration import ConfigurationManager
from src.finance_ml.component.model_prediction import ModelPrediction
# Assuming logger is configured elsewhere or imported from src.finance_ml
import logging

logger = logging.getLogger(__name__) # Use the configured logger


STAGE_NAME = "Model Prediction stage"

class ModelPredictionPipeline:
    def __init__(self):
        pass

    def main(self):
        try:
            logger.info(f">>>>>> stage {STAGE_NAME} started <<<<<<")
            # Get configuration
            config_manager = ConfigurationManager()
            model_prediction_config = config_manager.get_model_prediction_config()

            # Perform model prediction
            model_prediction = ModelPrediction(config=model_prediction_config)
            model_prediction.predict()

            logger.info(f">>>>>> stage {STAGE_NAME} completed <<<<<<\n\nx==========x")

        except Exception as e:
            logger.exception(e)
            raise e

# Example of how to run this stage
if __name__ == '__main__':
    try:
        # Assuming logging is configured at the entry point of the application
        # or configure it here if running this script directly for testing
        # logging.basicConfig(level=logging.INFO, format='[%(asctime)s: %(levelname)s: %(name)s: %(message)s]')

        obj = ModelPredictionPipeline()
        obj.main()
    except Exception as e:
        logger.exception(e)